## How to Run the Code
- Install requirements: `pip install -r requirements.txt`
- Run tests: `pytest test_final.py`
- Example usage is provided in this notebook.

## Bibliography
- https://numpy.org/
- https://matplotlib.org/


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.append("/workspaces/CP1-24-final-Forked")
from Tolani4.Code.final import sine_wave, fit_sine_wave

# Update file paths to be relative and point to the data folder
csv_files = [
    "Data/LL001_sinewalk1.csv",
    "Data/LL002_sinewalk2.csv",
    "Data/LL003_sinewalk3.csv",
    "Data/LL004_sinewalk4.csv",
    "Data/LL005_sinewalk5.csv"
]
temperatures = [294.26, 294.26, 294.26, 294.26, 294.26]  # List of temperatures in Kelvin for each walk

# Define function to plot GPS motion
def plot_gps_motion(csv_files, temperatures):
    plt.figure(figsize=(10, 6))
    for i, file in enumerate(csv_files):
        # Load the data and clean it
        df = pd.read_csv(file)
        df['Latitude'] = df['Latitude'].fillna(method='ffill')
        df['Longitude'] = df['Longitude'].fillna(method='ffill')
        df['Time (s)'] = df['Time (s)'].interpolate()

        time = df['Time (s)'].values
        latitude = df['Latitude'].values
        longitude = df['Longitude'].values

        # Convert to meters (assuming some conversion function is provided)
        x = latitude * 111320  # Approximate conversion to meters
        y = longitude * 111320  # Approximate conversion to meters

        # Fit the sine wave
        popt = fit_sine_wave(time, x)  # Fit sine wave for x (latitude converted to meters)
        fitted_x = sine_wave(time, *popt)

        # Plot the GPS motion
        plt.plot(time, x, label=f"Walk {i+1} - Temp: {temperatures[i]} K")
        plt.plot(time, fitted_x, '--', label=f"Fit {i+1}")

    plt.xlabel('Time (s)')
    plt.ylabel('Latitude (m)')
    plt.title('GPS Motion for Each Walk with Nonlinear Fit')
    plt.legend(loc='best')
    plt.show()

# Call the function to plot
plot_gps_motion(csv_files, temperatures)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/LL001_sinewalk1.csv'

<Figure size 1000x600 with 0 Axes>